# PedSen Initial ArchR Setup

In [1]:
quiet_library <- function(...) { suppressPackageStartupMessages(library(...)) }
quiet_library(ArchR)
quiet_library(hise)
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)
quiet_library(stringr)

In [2]:
addArchRThreads(32)
addArchRGenome("hg38")

Setting default number of Parallel threads to 32.

Setting default genome to Hg38.



# Setup

In [3]:
ped_arrows <- c('/home/jupyter/ped_vs_senior_arrow/B076-P1_PB00124-02.arrow',
                '/home/jupyter/ped_vs_senior_arrow/B069-P1_PB00172-02.arrow',
                '/home/jupyter/ped_vs_senior_arrow/B069-P1_PB00182-02.arrow',
                '/home/jupyter/ped_vs_senior_arrow/B076-P1_PB00127-02.arrow',
                '/home/jupyter/ped_vs_senior_arrow/B069-P1_PB00807-02.arrow',
                '/home/jupyter/ped_vs_senior_arrow/B065-P1_PB00173-02.arrow',
                '/home/jupyter/ped_vs_senior_arrow/B065-P1_PB00192-02.arrow',
                '/home/jupyter/ped_vs_senior_arrow/B065-P1_PB00197-02.arrow')

In [4]:
sen_arrows <- c('/home/jupyter/ped_vs_senior_arrow/B065-P1_PB00593-04.arrow',
                '/home/jupyter/ped_vs_senior_arrow/B069-P1_PB00323-02.arrow',
                '/home/jupyter/ped_vs_senior_arrow/B076-P1_PB00368-04.arrow',
                '/home/jupyter/ped_vs_senior_arrow/B076-P1_PB00353-03.arrow',
                '/home/jupyter/ped_vs_senior_arrow/B076-P1_PB00334-03.arrow',
                '/home/jupyter/ped_vs_senior_arrow/B076-P1_PB00377-03.arrow',
                '/home/jupyter/ped_vs_senior_arrow/B076-P1_PB00545-02.arrow',
                '/home/jupyter/ped_vs_senior_arrow/B076-P1_PB00599-02.arrow')

## Create ArchR Project

In [5]:
proj <- ArchRProject(
  ArrowFiles = c(ped_arrows, sen_arrows), 
  outputDirectory = "PedSen_ATAC",
  copyArrows = TRUE #This is recommened so that if you modify the Arrow files you have an original copy for later usage.
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

Validating Arrows...

Getting SampleNames...



Copying ArrowFiles to Ouptut Directory! If you want to save disk space set copyArrows = FALSE

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 
14 
15 
16 


Getting Cell Metadata...



Merging Cell Metadata...

Initializing ArchRProject...


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /_

In [6]:
proj


           ___      .______        ______  __    __  .______      
          /   \     |   _  \      /      ||  |  |  | |   _  \     
         /  ^  \    |  |_)  |    |  ,----'|  |__|  | |  |_)  |    
        /  /_\  \   |      /     |  |     |   __   | |      /     
       /  _____  \  |  |\  \\___ |  `----.|  |  |  | |  |\  \\___.
      /__/     \__\ | _| `._____| \______||__|  |__| | _| `._____|
    



class: ArchRProject 
outputDirectory: /home/jupyter/UPenn_Demo/PedSen_ATAC 
samples(16): B076-P1_PB00124-02 B069-P1_PB00172-02 ...
  B076-P1_PB00545-02 B076-P1_PB00599-02
sampleColData names(1): ArrowFiles
cellColData names(13): Sample TSSEnrichment ... nDiFrags BlacklistRatio
numberOfCells(1): 327662
medianTSS(1): 25.344
medianFrags(1): 6942

## Subset ArchR Project by barcodes from RNA+ADT object

In [7]:
rna_adt_so <- readRDS(file = '/home/jupyter/UPenn_Demo/sp_merge_gating_celltype.rds')

In [8]:
head(proj$cellNames)
clean_barcodes <- str_sub(proj$cellNames, 20, 51)
head(clean_barcodes)

[1] "B076-P1_PB00124-02#dcda7bf8e84e11eb939942010a19c839"
[2] "B076-P1_PB00124-02#8ff31584e84e11ebac5942010a19c839"
[3] "B076-P1_PB00124-02#13cd8d1ce84f11eb915942010a19c839"
[4] "B076-P1_PB00124-02#a8a121c0e84e11eb8ae942010a19c839"
[5] "B076-P1_PB00124-02#ab136800e84e11eb8ae942010a19c839"
[6] "B076-P1_PB00124-02#e9c95406e84e11ebbb6c42010a19c839"

[1] "dcda7bf8e84e11eb939942010a19c839" "8ff31584e84e11ebac5942010a19c839"
[3] "13cd8d1ce84f11eb915942010a19c839" "a8a121c0e84e11eb8ae942010a19c839"
[5] "ab136800e84e11eb8ae942010a19c839" "e9c95406e84e11ebbb6c42010a19c839"

In [9]:
proj$cellNames_clean <- clean_barcodes

In [10]:
barcodes_rna <- colnames(rna_adt_so[['RNA']])
head(barcodes_rna)

[1] "970c3f98e40811eba89d42010a19c839" "971b0bd6e40811eba89d42010a19c839"
[3] "97267fdee40811eba89d42010a19c839" "9728e26ae40811eba89d42010a19c839"
[5] "97475100e40811eba89d42010a19c839" "97789738e40811eba89d42010a19c839"

In [11]:
idx_new <- which(proj$cellNames_clean %in% barcodes_rna)
head(idx_new)

[1]  2  9 13 25 30 32

In [12]:
length(colnames(rna_adt_so[['RNA']]))
length(proj$cellNames)

[1] 273866

[1] 327662

In [13]:
cells_subset <- proj$cellNames[idx_new]
proj_subset <- proj[cells_subset,]
proj_subset


           ___      .______        ______  __    __  .______      
          /   \     |   _  \      /      ||  |  |  | |   _  \     
         /  ^  \    |  |_)  |    |  ,----'|  |__|  | |  |_)  |    
        /  /_\  \   |      /     |  |     |   __   | |      /     
       /  _____  \  |  |\  \\___ |  `----.|  |  |  | |  |\  \\___.
      /__/     \__\ | _| `._____| \______||__|  |__| | _| `._____|
    



class: ArchRProject 
outputDirectory: /home/jupyter/UPenn_Demo/PedSen_ATAC 
samples(16): B076-P1_PB00124-02 B069-P1_PB00172-02 ...
  B076-P1_PB00545-02 B076-P1_PB00599-02
sampleColData names(1): ArrowFiles
cellColData names(14): Sample TSSEnrichment ... BlacklistRatio
  cellNames_clean
numberOfCells(1): 273866
medianTSS(1): 25.301
medianFrags(1): 6923

## Transfer Metadata to ArchR Project

In [14]:
rna_metadata <- rna_adt_so@meta.data
head(rna_metadata)
head(proj_subset$cellNames_clean)

,orig.ident,nCount_RNA,nFeature_RNA,barcodes,adt_qc_flag,adt_umis,batch_id,cell_name,chip_id,hto_barcode,⋯,CMV,subject_id,age_cmv,nCount_clean_gex,nFeature_clean_gex,nCount_clean_adt,nFeature_clean_adt,SCT.weight,clean_adt.weight,Tiles.weight
,<chr>,<dbl>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,⋯,<fct>,<fct>,<chr>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
970c3f98e40811eba89d42010a19c839,SeuratProject,2338,1227,970c3f98e40811eba89d42010a19c839,Good,2184,B065,equatorial_wornout_skimmer,B065-P1C1,CTCCTCTGCAATTAC,⋯,Positive,UP1006,Pediatric_CMV-Positive,2338,1227,2121,33,0.3747029,0.3696419,0.2556552
971b0bd6e40811eba89d42010a19c839,SeuratProject,4382,1954,971b0bd6e40811eba89d42010a19c839,Good,3075,B065,artycrafty_graceful_robberfly,B065-P1C1,CTCCTCTGCAATTAC,⋯,Positive,UP1006,Pediatric_CMV-Positive,4380,1952,3013,32,0.4311530,0.3859967,0.1828503
97267fdee40811eba89d42010a19c839,SeuratProject,4232,1932,97267fdee40811eba89d42010a19c839,Good,2234,B065,nonpsychic_condemned_mutt,B065-P1C1,CTCCTCTGCAATTAC,⋯,Positive,UP1006,Pediatric_CMV-Positive,4232,1932,2186,29,0.2557901,0.4349854,0.3092244
9728e26ae40811eba89d42010a19c839,SeuratProject,3515,1693,9728e26ae40811eba89d42010a19c839,Good,3204,B065,contiguous_ultrasonic_gavial,B065-P1C1,CTCCTCTGCAATTAC,⋯,Positive,UP1006,Pediatric_CMV-Positive,3513,1692,3145,29,0.3379603,0.3827271,0.2793125
97475100e40811eba89d42010a19c839,SeuratProject,3021,1464,97475100e40811eba89d42010a19c839,Good,2798,B065,late_oriental_adouri,B065-P1C1,CTCCTCTGCAATTAC,⋯,Positive,UP1006,Pediatric_CMV-Positive,3020,1463,2734,32,0.3000353,0.4466715,0.2532933
97789738e40811eba89d42010a19c839,SeuratProject,3717,1701,97789738e40811eba89d42010a19c839,Good,2799,B065,seclusive_sandstone_manta,B065-P1C1,CTCCTCTGCAATTAC,⋯,Positive,UP1006,Pediatric_CMV-Positive,3715,1700,2709,31,0.3359731,0.2803253,0.3837016


[1] "8ff31584e84e11ebac5942010a19c839" "682dcba2e84e11eb975f42010a19c839"
[3] "0ff9ba76e84f11eb915942010a19c839" "c1f529e6e84e11eb909942010a19c839"
[5] "8551ed4ee84e11eb8a7c42010a19c839" "679178bae84e11eb975f42010a19c839"

In [15]:
bc_idx <- barcodes_rna %in% clean_barcodes
table(bc_idx)

bc_idx
  TRUE 
273866 

In [16]:
# all cells overlap - so no need to subset rna metadata
#rna_meta_subset <- rna_metadata[bc_idx,]
#length(rownames(rna_meta_subset))

In [17]:
# check barcode alignment
table(proj_subset$cellNames_clean == rownames(rna_metadata))


 FALSE   TRUE 
273863      3 

In [18]:
head(rna_metadata)

,orig.ident,nCount_RNA,nFeature_RNA,barcodes,adt_qc_flag,adt_umis,batch_id,cell_name,chip_id,hto_barcode,⋯,CMV,subject_id,age_cmv,nCount_clean_gex,nFeature_clean_gex,nCount_clean_adt,nFeature_clean_adt,SCT.weight,clean_adt.weight,Tiles.weight
,<chr>,<dbl>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,⋯,<fct>,<fct>,<chr>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
970c3f98e40811eba89d42010a19c839,SeuratProject,2338,1227,970c3f98e40811eba89d42010a19c839,Good,2184,B065,equatorial_wornout_skimmer,B065-P1C1,CTCCTCTGCAATTAC,⋯,Positive,UP1006,Pediatric_CMV-Positive,2338,1227,2121,33,0.3747029,0.3696419,0.2556552
971b0bd6e40811eba89d42010a19c839,SeuratProject,4382,1954,971b0bd6e40811eba89d42010a19c839,Good,3075,B065,artycrafty_graceful_robberfly,B065-P1C1,CTCCTCTGCAATTAC,⋯,Positive,UP1006,Pediatric_CMV-Positive,4380,1952,3013,32,0.4311530,0.3859967,0.1828503
97267fdee40811eba89d42010a19c839,SeuratProject,4232,1932,97267fdee40811eba89d42010a19c839,Good,2234,B065,nonpsychic_condemned_mutt,B065-P1C1,CTCCTCTGCAATTAC,⋯,Positive,UP1006,Pediatric_CMV-Positive,4232,1932,2186,29,0.2557901,0.4349854,0.3092244
9728e26ae40811eba89d42010a19c839,SeuratProject,3515,1693,9728e26ae40811eba89d42010a19c839,Good,3204,B065,contiguous_ultrasonic_gavial,B065-P1C1,CTCCTCTGCAATTAC,⋯,Positive,UP1006,Pediatric_CMV-Positive,3513,1692,3145,29,0.3379603,0.3827271,0.2793125
97475100e40811eba89d42010a19c839,SeuratProject,3021,1464,97475100e40811eba89d42010a19c839,Good,2798,B065,late_oriental_adouri,B065-P1C1,CTCCTCTGCAATTAC,⋯,Positive,UP1006,Pediatric_CMV-Positive,3020,1463,2734,32,0.3000353,0.4466715,0.2532933
97789738e40811eba89d42010a19c839,SeuratProject,3717,1701,97789738e40811eba89d42010a19c839,Good,2799,B065,seclusive_sandstone_manta,B065-P1C1,CTCCTCTGCAATTAC,⋯,Positive,UP1006,Pediatric_CMV-Positive,3715,1700,2709,31,0.3359731,0.2803253,0.3837016


In [19]:
rna_metadata <- rna_metadata[match(proj_subset$cellNames_clean, rownames(rna_metadata)),]

In [20]:
head(rna_metadata)
head(proj_subset$cellNames_clean)

,orig.ident,nCount_RNA,nFeature_RNA,barcodes,adt_qc_flag,adt_umis,batch_id,cell_name,chip_id,hto_barcode,⋯,CMV,subject_id,age_cmv,nCount_clean_gex,nFeature_clean_gex,nCount_clean_adt,nFeature_clean_adt,SCT.weight,clean_adt.weight,Tiles.weight
,<chr>,<dbl>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,⋯,<fct>,<fct>,<chr>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
8ff31584e84e11ebac5942010a19c839,SeuratProject,3838,1868,8ff31584e84e11ebac5942010a19c839,Good,2901,B076,vulcanian_economic_whelp,B076-P1C1,CTCCTCTGCAATTAC,⋯,Positive,UP1001,Pediatric_CMV-Positive,3833,1866,2712,37,0.9054187,2.719901e-02,6.738231e-02
682dcba2e84e11eb975f42010a19c839,SeuratProject,4416,1934,682dcba2e84e11eb975f42010a19c839,Good,4040,B076,occasional_final_drongo,B076-P1C1,CTCCTCTGCAATTAC,⋯,Positive,UP1001,Pediatric_CMV-Positive,4413,1932,3701,35,0.9999102,7.899506e-05,1.081362e-05
0ff9ba76e84f11eb915942010a19c839,SeuratProject,8643,3197,0ff9ba76e84f11eb915942010a19c839,Good,2859,B076,diagnostic_repayable_porpoise,B076-P1C2,CTCCTCTGCAATTAC,⋯,Positive,UP1001,Pediatric_CMV-Positive,8638,3195,2691,36,0.7058793,1.444748e-01,1.496458e-01
c1f529e6e84e11eb909942010a19c839,SeuratProject,19523,4902,c1f529e6e84e11eb909942010a19c839,Good,4145,B076,mean_postal_gourami,B076-P1C2,CTCCTCTGCAATTAC,⋯,Positive,UP1001,Pediatric_CMV-Positive,19507,4900,3988,38,0.2565166,2.757653e-01,4.677182e-01
8551ed4ee84e11eb8a7c42010a19c839,SeuratProject,7758,3180,8551ed4ee84e11eb8a7c42010a19c839,Good,1670,B076,lowfat_pelage_aracari,B076-P1C1,CTCCTCTGCAATTAC,⋯,Positive,UP1001,Pediatric_CMV-Positive,7752,3178,1535,37,0.1834026,1.923982e-01,6.241992e-01
679178bae84e11eb975f42010a19c839,SeuratProject,5114,2364,679178bae84e11eb975f42010a19c839,Good,2293,B076,muscovite_pegmatite_beagle,B076-P1C1,CTCCTCTGCAATTAC,⋯,Positive,UP1001,Pediatric_CMV-Positive,5113,2363,2161,36,0.1447526,2.035559e-01,6.516916e-01


[1] "8ff31584e84e11ebac5942010a19c839" "682dcba2e84e11eb975f42010a19c839"
[3] "0ff9ba76e84f11eb915942010a19c839" "c1f529e6e84e11eb909942010a19c839"
[5] "8551ed4ee84e11eb8a7c42010a19c839" "679178bae84e11eb975f42010a19c839"

In [21]:
# check re-aligned barcodes - should be all true
table(proj_subset$cellNames_clean == rownames(rna_metadata))


  TRUE 
273866 

### Add metadata

In [22]:
colnames(rna_metadata)

[1] "orig.ident"                             
 [2] "nCount_RNA"                             
 [3] "nFeature_RNA"                           
 [4] "barcodes"                               
 [5] "adt_qc_flag"                            
 [6] "adt_umis"                               
 [7] "batch_id"                               
 [8] "cell_name"                              
 [9] "chip_id"                                
[10] "hto_barcode"                            
[11] "hto_category"                           
[12] "n_genes"                                
[13] "n_mito_umis"                            
[14] "n_reads"                                
[15] "n_umis"                                 
[16] "original_barcodes"                      
[17] "pbmc_sample_id"                         
[18] "pool_id"                                
[19] "rna_cell_uuid"                          
[20] "seurat_pbmc_type"                       
[21] "seurat_pbmc_type_score"                 
[22] "umap_1"                                 
[23] "umap_2"                                 
[24] "well_id"                                
[25] "nCount_ADT"                             
[26] "nFeature_ADT"                           
[27] "nCount_Tiles"                           
[28] "nFeature_Tiles"                         
[29] "percent.mt"                             
[30] "nCount_SCT"                             
[31] "nFeature_SCT"                           
[32] "SCT_snn_res.0.5"                        
[33] "seurat_clusters"                        
[34] "predicted.celltype.l1.score"            
[35] "predicted.celltype.l1"                  
[36] "predicted.celltype.l2.score"            
[37] "predicted.celltype.l2"                  
[38] "predicted.celltype.l3.score"            
[39] "predicted.celltype.l3"                  
[40] "predicted.t_celltype.l1.score"          
[41] "predicted.t_celltype.l1"                
[42] "predicted.t_celltype.l2.score"          
[43] "predicted.t_celltype.l2"                
[44] "predicted.t_celltype.l3.score"          
[45] "predicted.t_celltype.l3"                
[46] "pediatric_senior"                       
[47] "celltype_l0"                            
[48] "nCount_prediction.score.celltype.l1"    
[49] "nFeature_prediction.score.celltype.l1"  
[50] "nCount_prediction.score.celltype.l2"    
[51] "nFeature_prediction.score.celltype.l2"  
[52] "nCount_prediction.score.celltype.l3"    
[53] "nFeature_prediction.score.celltype.l3"  
[54] "nCount_predicted_ADT"                   
[55] "nFeature_predicted_ADT"                 
[56] "nCount_prediction.score.t_celltype.l1"  
[57] "nFeature_prediction.score.t_celltype.l1"
[58] "nCount_prediction.score.t_celltype.l2"  
[59] "nFeature_prediction.score.t_celltype.l2"
[60] "nCount_prediction.score.t_celltype.l3"  
[61] "nFeature_prediction.score.t_celltype.l3"
[62] "nCount_x10_ADT"                         
[63] "nFeature_x10_ADT"                       
[64] "gating_celltype"                        
[65] "CMV"                                    
[66] "subject_id"                             
[67] "age_cmv"                                
[68] "nCount_clean_gex"                       
[69] "nFeature_clean_gex"                     
[70] "nCount_clean_adt"                       
[71] "nFeature_clean_adt"                     
[72] "SCT.weight"                             
[73] "clean_adt.weight"                       
[74] "Tiles.weight"

In [23]:
proj_subset$gating_celltype <- rna_metadata$gating_celltype
proj_subset$pediatric_senior <- rna_metadata$pediatric_senior
proj_subset$age_cmv <- rna_metadata$age_cmv
proj_subset$predicted.t_celltype.l1 <- rna_metadata$predicted.t_celltype.l1
proj_subset$predicted.t_celltype.l2 <- rna_metadata$predicted.t_celltype.l2
proj_subset$predicted.t_celltype.l3 <- rna_metadata$predicted.t_celltype.l3
proj_subset$batch_id <- rna_metadata$batch_id
proj_subset$pbmc_sample_id <- rna_metadata$pbmc_sample_id

## Add Gene Expression to ArchR Project

In [24]:
library(SingleCellExperiment)

In [25]:
rna_mtx <- as.sparse(rna_adt_so@assays$RNA@counts)

In [26]:
head(colnames(rna_mtx))
head(proj$cellNames)

[1] "970c3f98e40811eba89d42010a19c839" "971b0bd6e40811eba89d42010a19c839"
[3] "97267fdee40811eba89d42010a19c839" "9728e26ae40811eba89d42010a19c839"
[5] "97475100e40811eba89d42010a19c839" "97789738e40811eba89d42010a19c839"

[1] "B076-P1_PB00124-02#dcda7bf8e84e11eb939942010a19c839"
[2] "B076-P1_PB00124-02#8ff31584e84e11ebac5942010a19c839"
[3] "B076-P1_PB00124-02#13cd8d1ce84f11eb915942010a19c839"
[4] "B076-P1_PB00124-02#a8a121c0e84e11eb8ae942010a19c839"
[5] "B076-P1_PB00124-02#ab136800e84e11eb8ae942010a19c839"
[6] "B076-P1_PB00124-02#e9c95406e84e11ebbb6c42010a19c839"

In [27]:
archr_style_barcodes <- paste(rna_adt_so$batch_id, "-P1_",rna_adt_so$pbmc_sample_id,"#",colnames(rna_mtx), sep = "")
head(archr_style_barcodes)

[1] "B065-P1_PB00173-02#970c3f98e40811eba89d42010a19c839"
[2] "B065-P1_PB00173-02#971b0bd6e40811eba89d42010a19c839"
[3] "B065-P1_PB00173-02#97267fdee40811eba89d42010a19c839"
[4] "B065-P1_PB00173-02#9728e26ae40811eba89d42010a19c839"
[5] "B065-P1_PB00173-02#97475100e40811eba89d42010a19c839"
[6] "B065-P1_PB00173-02#97789738e40811eba89d42010a19c839"

In [28]:
table(archr_style_barcodes %in% proj$cellNames)
rna_adt_so$ATAC_Cell_ID <- archr_style_barcodes


  TRUE 
273866 

In [29]:
colnames(rna_mtx) <- archr_style_barcodes

In [30]:
atac_genes <- getGenes(ArchRProj = proj)
head(atac_genes)

GRanges object with 6 ranges and 2 metadata columns:
      seqnames        ranges strand |     gene_id       symbol
         <Rle>     <IRanges>  <Rle> | <character>  <character>
  [1]     chr1   69091-70008      + |       79501        OR4F5
  [2]     chr1 139790-140339      - |      729737    LOC729737
  [3]     chr1 585990-629010      - |   101928626 LOC101928626
  [4]     chr1 632325-632413      - |   102465432         <NA>
  [5]     chr1 817371-819837      + |      400728       FAM87B
  [6]     chr1 825138-859446      + |      643837    LINC01128
  -------
  seqinfo: 24 sequences from hg38 genome

In [31]:
genes <- getGenes(proj, symbols = rownames(rna_mtx))

In [32]:
genes

GRanges object with 21406 ranges and 2 metadata columns:
          seqnames            ranges strand |     gene_id      symbol
             <Rle>         <IRanges>  <Rle> | <character> <character>
      [1]     chr1       69091-70008      + |       79501       OR4F5
      [2]     chr1     817371-819837      + |      400728      FAM87B
      [3]     chr1     825138-859446      + |      643837   LINC01128
      [4]     chr1     826206-827522      - |       79854   LINC00115
      [5]     chr1     868071-876903      - |      284593      FAM41C
      ...      ...               ...    ... .         ...         ...
  [21402]     chrY 21583600-21594666      - |       83868      TTTY13
  [21403]     chrY 21903618-21918067      - |      378949      RBMY1D
  [21404]     chrY 22296798-22298876      - |       83863       TTTY5
  [21405]     chrY 23129355-23199094      - |        1617        DAZ1
  [21406]     chrY 23219447-23291356      + |       57055        DAZ2
  -------
  seqinfo: 24 sequences

In [33]:
idx <- rownames(rna_mtx) %in% atac_genes$symbol
head(idx)

[1] FALSE FALSE  TRUE FALSE FALSE FALSE

In [34]:
rna_mtx_subset <- rna_mtx[idx,]

In [35]:
sce <- SingleCellExperiment(list(counts = rna_mtx_subset),rowRanges= genes)
sce

class: SingleCellExperiment 
dim: 21406 273866 
metadata(0):
assays(1): counts
rownames(21406): OR4F5 FAM87B ... DAZ1 DAZ2
rowData names(2): gene_id symbol
colnames(273866): B065-P1_PB00173-02#970c3f98e40811eba89d42010a19c839
  B065-P1_PB00173-02#971b0bd6e40811eba89d42010a19c839 ...
  B076-P1_PB00599-02#145ac9fce84f11eb915942010a19c839
  B076-P1_PB00599-02#148df6b0e84f11eb915942010a19c839
colData names(0):
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

In [36]:
proj_subset <- addGeneExpressionMatrix(input = proj_subset, seRNA = sce)

ArchR logging to : ArchRLogs/ArchR-addGeneExpressionMatrix-1d0e5c8eee-Date-2022-03-02_Time-20-57-55.log
If there is an issue, please report to github with logFile!

Overlap w/ scATAC = 1

2022-03-02 20:57:59 : 

Overlap Per Sample w/ scATAC : B065-P1_PB00173-02=10985,B065-P1_PB00192-02=11266,B065-P1_PB00197-02=11004,B065-P1_PB00593-04=9849,B069-P1_PB00172-02=7967,B069-P1_PB00182-02=10950,B069-P1_PB00323-02=12225,B069-P1_PB00807-02=10533,B076-P1_PB00124-02=25227,B076-P1_PB00127-02=26841,B076-P1_PB00334-03=24174,B076-P1_PB00353-03=23315,B076-P1_PB00368-04=21978,B076-P1_PB00377-03=25223,B076-P1_PB00545-02=20380,B076-P1_PB00599-02=21949

2022-03-02 20:58:00 : 

2022-03-02 20:58:46 : Batch Execution w/ safelapply!, 0 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-addGeneExpressionMatrix-1d0e5c8eee-Date-2022-03-02_Time-20-57-55.log



# Save ArchR Project

In [37]:
saveArchRProject(ArchRProj = proj_subset, outputDirectory = '/home/jupyter/UPenn_Demo/PedSen_ATAC/', load = F)

Saving ArchRProject...



In [38]:
getAvailableMatrices(proj_subset)

[1] "GeneExpressionMatrix" "GeneScoreMatrix"      "TileMatrix"

In [39]:
sessionInfo()

R version 4.1.2 (2021-11-01)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 20.04.3 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/liblapack.so.3

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] parallel  stats4    stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] SingleCellExperiment_1.14.1       BSgenome.Hsapiens.UCSC.hg38_1.4.3
 [3] BSgenome_1.60.0                   rtracklayer_1.52.1               
 [5] Biostrings_2.60.2                 XVector_0.32.0                   
 [7] stringr_1.4.0                     Nebulosa_1.2.0                